In [1]:
import datetime

from cogment_lab.actors import RandomActor, ConstantActor
from cogment_lab.envs.pettingzoo import AECEnvironment
from cogment_lab.process_manager import Cogment
from cogment_lab.utils.runners import process_cleanup


In [2]:
# Cleans up potentially hanging background processes from previous runs
process_cleanup()

Processes terminated successfully.


In [3]:
logpath = f"logs/logs-{datetime.datetime.now().isoformat()}"

cog = Cogment(log_dir=logpath)

print(logpath)

logs/logs-2023-12-13T19:23:09.453551


/Users/ariel/PycharmProjects/cogment_lab/venv/lib/python3.10/site-packages/cogment/context.py:213: UserWarning: No logging handler defined (e.g. logging.basicConfig)
  warnings.warn("No logging handler defined (e.g. logging.basicConfig)")


In [4]:
# Launch an environment in a subprocess

cenv = AECEnvironment(env_path="cogment_lab.envs.conversions.observer.GymObserverAECAEC",
                      make_kwargs={"gym_env_name": "LunarLander-v2"},
                      render=True)

await cog.run_env(env=cenv, 
                  env_name="lunar",
                  port=9011, 
                  log_file="env.log")


True

In [5]:
# Launch two dummy actors in subprocesses

random_actor = RandomActor(cenv.env.action_space("gym"))
constant_actor = ConstantActor(0)

await cog.run_actor(actor=random_actor, 
                    actor_name="random", 
                    port=9021, 
                    log_file="actor-random.log")

await cog.run_actor(actor=constant_actor,
                    actor_name="constant",
                    port=9022,
                    log_file="actor-constant.log")


True

In [6]:
# Check what's running

cog.processes

{'lunar': <SpawnProcess name='SpawnProcess-1' pid=17412 parent=17393 started>,
 'random': <SpawnProcess name='SpawnProcess-2' pid=17417 parent=17393 started>,
 'constant': <SpawnProcess name='SpawnProcess-3' pid=17421 parent=17393 started>}

In [7]:
LUNAR_LANDER_ACTIONS = ["no-op", "ArrowRight", "ArrowUp", "ArrowLeft"]

# Change this if you use a different environment. Only discrete actions are supported for now.

actions = LUNAR_LANDER_ACTIONS
await cog.run_web_ui(actions=actions, log_file="human.log", fps=60)

True

In [8]:
# Get data from a random + random trial
# You can change the values in `actor_impls` between `web_ui`, `random`, and `constant` to see the different behaviors

trial_id = await cog.start_trial(
    env_name="lunar",
    session_config={"render": True},
    actor_impls={
        "gym": "random",
        "observer": "web_ui",
    },
)

In [9]:
data = await cog.get_trial_data(trial_id)

In [13]:
await cog.cleanup()